In [1]:
using DelimitedFiles
# using Statistics
using DataFrames
using CSV
using Printf
using DataStructures
using Base.Threads
using ProgressMeter
using NPZ
using MMCAcovid19
using ArgParse
using JSON
using Dates

In [22]:
## -----------------------------------------------------------------------------
## LOADING DATA
## -----------------------------------------------------------------------------

data_path = "../data/"
instance_path = "../data/"

function load_params(data_path::String, instance_path::String)::Tuple{Epidemic_Params, Population_Params}
    # Config file
    config = JSON.parsefile(joinpath(data_path, "config.json"))

    # Parameters to simulate
    paramsDF = CSV.read(joinpath(instance_path, "params.csv"), DataFrame)

    # Output simulation
    output_path = joinpath(instance_path, "output")

    # Loading mobility network
    network = CSV.read(joinpath(data_path, "R_mobility_matrix.csv"), DataFrame)
    edgelist = convert(Array{Int64, 2}, network[:, 1:2])
    Rᵢⱼ = copy(network[:, 3])

    # Patch surface
    sᵢ = CSV.read(joinpath(data_path, "S_patch_surface_area.csv"), DataFrame)[:,"area"]

    # Population info
    nᵢ_ages = CSV.read(@sprintf("%s/n_population_patch_age.csv", data_path), DataFrame);
    A0_instance_filename = @sprintf("%s/%s", instance_path, config["A0_filename"])
    A0_data_filename = @sprintf("%s/%s", data_path, config["A0_filename"])
    if isfile(A0_instance_filename)
      println("Loading A0 from instance/ folder")
      conditions₀ = CSV.read(A0_instance_filename, DataFrame)
    else
      println("Loading A0 from data/ folder")
      conditions₀ = CSV.read(A0_data_filename, DataFrame)
    end

    C = readdlm(@sprintf("%s/C_age_contact_matrix.csv", data_path),
                ',', Float64)

    # Containement measures
    κ₀_df = CSV.read(@sprintf("%s/%s", data_path, config["kappa0_filename"]), DataFrame)

    # Patch to CCAA mapping matrix
    PatchToCCAA = npzread(@sprintf("%s/patch_to_ccaa.npy", data_path))
    n_ccaa = size(PatchToCCAA)[1]
    n_patches = size(PatchToCCAA)[2]


    ## -----------------------------------------------------------------------------
    ## SETTING PARAMETERS (some of them are just placeholders for the values on DF)
    ## -----------------------------------------------------------------------------


    ## POPULATION

    # Patch population for each strata
    nᵢᵍ = copy(transpose(convert(Array{Float64,2}, nᵢ_ages[:,["Y", "M", "O"]])))

    # Total patch population
    nᵢ = convert(Array{Float64,1}, nᵢ_ages[:,"Total"])

    # Num. of patches
    M = length(nᵢ)

    # Num of stratas
    G = size(C)[1]

    # Average number of contacts per strata
    kᵍ = [11.8, 13.3, 6.76]

    # Average number of contacts at home per strata
    kᵍ_h = [3.15, 3.17, 3.28]

    # Average number of contacts at work per strata
    kᵍ_w = [1.72, 5.18, 0.0]

    # Degree of mobility per strata
    pᵍ = [0.0, 1.0, 0.00]

    # Density factor
    ξ = 0.01

    # Average household size
    σ = 2.5

    # Check network structure and self-loop correction
    edgelist, Rᵢⱼ = MMCAcovid19.correct_self_loops(edgelist, Rᵢⱼ, M)


    # population_params = Population_Params(G, M, nᵢᵍ, kᵍ, kᵍ_h, kᵍ_w, C, pᵍ, edgelist, Rᵢⱼ, sᵢ, ξ, σ)


    ## EPIDEMIC PARAMETERS

    # Scaling of the asymptomatic infectivity
    scale_β = 0.51

    # Infectivity of Symptomatic
    βᴵ = 0.0903

    # Infectivity of Asymptomatic
    βᴬ = scale_β * βᴵ

    # Exposed rate
    ηᵍ = [1/3.64, 1/3.64, 1/3.64]

    # Asymptomatic rate
    αᵍ = [1/3.76, 1/1.56, 1/1.56]

    # Infectious rate
    μᵍ = [1/1.0, 1/3.2, 1/3.2]

    # Direct death probability
    θᵍ = [0.00, 0.008, 0.047]

    # Direct death probability
    γᵍ = [0.0003, 0.003, 0.026]

    # Pre-deceased rate
    ζᵍ = [1/7.8, 1/7.8, 1/7.8]

    # Pre-hospitalized in ICU rate
    λᵍ = ones(Float64, 3)

    # Fatality probability in ICU
    ωᵍ = [0.30, 0.30, 0.30]

    # Death rate in ICU
    ψᵍ = [1/7.0, 1/7.0, 1/7.0]

    # ICU discharge rate
    χᵍ = [1/21.0, 1/21.0, 1/21.0]

    # Number of timesteps
    # dia inicial: 9 Feb
    # dia final: 14 Abril (66 dias)
    # T = 123
    # T = 66
    first_day = Date(config["first_day_simulation"])
    last_day = Date(config["last_day_simulation"])
    T = (last_day - first_day).value + 1

    # Epidemic parameters
    # epi_params = Epidemic_Params(βᴵ, βᴬ, ηᵍ, αᵍ, μᵍ, θᵍ, γᵍ, ζᵍ, λᵍ, ωᵍ, ψᵍ, χᵍ, G, M, T)

    ## CONFINEMENT MEASURES

    # Timesteps when the containment measures will be applied
    tᶜs = κ₀_df.time[:]

    # Array of level of confinement
    κ₀s = κ₀_df.reduction[:]

    # Array of premeabilities of confined households
    ϕs = ones(Float64, length(tᶜs))

    # Array of social distancing measures
    δs = ones(Float64, length(tᶜs))


    ## INITIALIZATION OF THE EPIDEMICS

    # Initial number of exposed individuals
    E₀ = zeros(G, M)

    # Initial number of infectious asymptomatic individuals
    A₀ = zeros(Float64, G, M)

    # Distribution of the intial infected individuals per strata
    # WARN: ni idea de por que es necesario el .+ 1 aqui (las semillas ya vienen indexadas partiendo de 1) pero si no lo pongo la simulacion simplemente no funciona
    A₀[1, Int.(conditions₀[:,"idx"])] .= 0.12 .* conditions₀[:,"seed"]
    A₀[2, Int.(conditions₀[:,"idx"])] .= 0.16 .* conditions₀[:,"seed"]
    A₀[3, Int.(conditions₀[:,"idx"])] .= 0.72 .* conditions₀[:,"seed"]

    # Initial number of infectious symptomatic individuals
    I₀ = zeros(Float64, G, M)
    
    epi_params = Epidemic_Params(βᴵ, βᴬ, ηᵍ, αᵍ, μᵍ, θᵍ, γᵍ, ζᵍ, λᵍ, ωᵍ, ψᵍ, χᵍ, G, M, T)
    pop_params = Population_Params(G, M, nᵢᵍ, kᵍ, kᵍ_h, kᵍ_w, C, pᵍ, edgelist, Rᵢⱼ, sᵢ, ξ, σ)
    
    return (epi_params, pop_params)
end

epi_params, pop_params = load_params(data_path, instance_path)

Loading A0 from instance/ folder


(Epidemic_Params([0.0903], [0.046053000000000004], [0.2747252747252747, 0.2747252747252747, 0.2747252747252747], [0.26595744680851063, 0.641025641025641, 0.641025641025641], [1.0, 0.3125, 0.3125], [0.0, 0.008, 0.047], [0.0003, 0.003, 0.026], [0.12820512820512822, 0.12820512820512822, 0.12820512820512822], [1.0, 1.0, 1.0], [0.3, 0.3, 0.3], [0.14285714285714285, 0.14285714285714285, 0.14285714285714285], [0.047619047619047616, 0.047619047619047616, 0.047619047619047616], 367, [1.0 1.0 … 1.0 1.0; 1.0 1.0 … 1.0 1.0; 1.0 1.0 … 1.0 1.0]

[1.0 1.0 … 1.0 1.0; 1.0 1.0 … 1.0 1.0; 1.0 1.0 … 1.0 1.0]

[1.0 1.0 … 1.0 1.0; 1.0 1.0 … 1.0 1.0; 1.0 1.0 … 1.0 1.0]

...

[1.0 1.0 … 1.0 1.0; 1.0 1.0 … 1.0 1.0; 1.0 1.0 … 1.0 1.0]

[1.0 1.0 … 1.0 1.0; 1.0 1.0 … 1.0 1.0; 1.0 1.0 … 1.0 1.0]

[1.0 1.0 … 1.0 1.0; 1.0 1.0 … 1.0 1.0; 1.0 1.0 … 1.0 1.0], [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]

[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]

[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; 0.0 0